# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Les images proviennent du jeu de données publiques *The Extended Yale Face Database B* disponible sur le site http://vision.ucsd.edu/~iskwak/ExtYaleDatabase/ExtYaleB.html.


## TD6 : Reconnaissance faciale avec l'analyse en composantes principales


Nous utiliserons des images récupérées de la base de données publique de Yale$^{(1)}$ que vous pouvez trouver sur Moodle. 

L'ensemble d'entraînement est composé de 28 images par individu pour 28 individus. 

L'ensemble de test est composé de 69 images non contenues dans l'ensemble d'entraînement.

Pour chaque image de l'ensemble de test, on souhaite déterminer s'il s'agit d'une personne connue dans l'ensemble d'entraînement ou s'il s'agit d'une personne inconnue de l'ensemble d'entraînement.

## Préambule

Plusieurs librairies que nous n'utilisons pas régulièrement dans le cours sont nécessaire. Exécutez la prochaine cellule de code pour installer ces librairies.

In [ ]:
import Pkg
Pkg.add(["Images", "Netpbm", "ImageMagick", "Colors"])

In [ ]:
# Librairies standards du cours
using Statistics, LinearAlgebra, Gadfly, DataFrames

# Librairie pour le traitement des images
using Images, Netpbm, ImageMagick, Colors

## Fonctions utiles

Voici une liste de fonctions qui vous seront utiles pour ce TD.
- `imgrayconvert` : permet de convertir une image en niveaux de gris pour chacun des pixels
- `imshow` : affiche la matrice d'intensité des niveaux de gris 

In [ ]:
"""
    imgrayconvert(imageFileName ; columnStack=true)

Conversion en intensité de gris de l'image du fichier `imageFileName`.

### Arguments
- `imageFileName::string` : le nom du fichier de l'image
- `columnStack::bool=true` : Si `true`, l'image est renvoyée comme un vecteur colonne (option par défaut) 
                             sinon la fonction renvoie la matrice des niveaux de gris.

### Details
 
La fonction retourne la matrice ou le vecteur colonne des niveaux de gris.
 
### Examples

\```
 julia> imgrayconvert(imageFileName)
 julia> imgrayconvert(imageFileName ; columnStack=false)
\```

"""
function imgrayconvert(imageFileName::String ; columnStack::Bool=false, T::DataType=UInt8)
    im = load(imageFileName)
    X = Float64.(im)
    if columnStack
        Y = X[:]
    else
        Y = X
    end
    return Y
end

In [ ]:
"""
    imshow(X::Array{<:Real,2}; colorscaling::Bool=false)

Affiche une matrice en une image composée de niveau de gris.

### Arguments
- `X::Array{<:Real,2}` : Une matrice d'intensité de gris dans l'intervalle [0,1].
- `colorscaling::Bool=false` : Ajustement des limites de l'échelles de couleur.


### Details

Lorsque `colorscaling = true`, l'échelle d'affichage des couleurs est ajustée de façon à ce que l'intensité minimale de l'image soit noir et l'intensité maximale soit blanc.
 
### Examples

\```
 julia> imshow(X)
 julia> imshow(X, colorscaling = true)
\```

"""

function imshow(X::Array{<:Real,2}; colorscaling::Bool=true)
    
    if colorscaling
        m = minimum(X)
        M = maximum(X)
    
        Z = (X .- m) / (M-m)
        Gray.(Z)
    else
        Gray.(X)
    end
    
end

"""
    imshow(x::Vector{<:Real}; im_size::Tuple{<:Int,<:Int64}, colorscaling::Bool=false)

Affiche vecteur en une image de dimension `im_size` composée de niveau de gris.

### Arguments
- `x::Vector{<:Real}` : Une matrice ou un vecteur colonne à afficher.
- `im_size::Tuple{<:Int,<:Int}` : Un tuple de Int indicant la taille de l'image.
- `colorscaling::Bool=false` : Ajustement des limites de l'échelles de couleur.


### Details

Lorsque `colorscaling = true`, l'échelle d'affichage des couleurs est ajustée de façon à ce que l'intensité minimale de l'image soit noir et l'intensité maximale soit blanc.
 
### Examples

\```
 julia> imshow(x, (m₁, m₂))
 julia> imshow(x, (m₁, m₂), colorscaling=true)
\```

"""
function imshow(x::Vector{<:Real}, im_size::Tuple{<:Int,<:Int64}; colorscaling::Bool=true)
    
    X = reshape(x, im_size)
    
    imshow(X, colorscaling=colorscaling)
    
end


# 1. Chargement des images d'entraînement et de test
___

Les images d'entraînement sont contenues dans le dossier *Train* du jeu de données que vous pouvez récupérer sur Moodle. L'ensemble d'entraînement est constitué de 784 images provenant de 28 personnes différentes, soit de 28 images par personne.

Les images sont des visages déjà correctement centrés, ce qui nous permet de nous concentrer directement sur la reconnaissance des visages.

In [ ]:
# Récupération de tous les noms de fichiers de l'échantillon d'entraînement
file = readdir("Train")
trainFileName = ["Train/"*file[i] for i=1:length(file)];

In [ ]:
# Nombre d'images dans l'ensemble d'entraînement
n = length(trainFileName)

In [ ]:
# Affichage des 8 photos de la première personne
load.(trainFileName[1:8])

In [ ]:
# Affichage des 8 photos de la deuxième personne
load.(trainFileName[29:36])

In [ ]:
# Récupération de tous les noms de fichiers de l'échantillon d'entraînement
file = readdir("Test")
testFileName = ["Test/"*file[i] for i=1:length(file)];

In [ ]:
# Affichage des 8 premières images de l'ensemble de test
load.(testFileName[1:8])

In [ ]:
length(testFileName)

### (a) Convertissez en matrice de niveaux de gris la première image

Utilisez pour ce faire la fonction `imgrayconvert` fournie.

### (b) Affichez la matrice des intensités de gris 

Utilisez la fonction `imshow` fournie.

### (c) Récupérez le nombre de lignes et le nombre de colonnes des images

Assignez les variables suivantes :
- `m₁` : le nombre de lignes d'une image;
- `m₂` : le nombre de colonnes d'une image ;
- `m` : le nombre total de pixels d'une image.

### (d) Emmagasinez toutes les images dans une seule matrice X.

Chaque ligne de la matrice X correspond à une image. 

De façon analogue à la régression, la matrice X possède n observations (le nombre d'images) avec m variables explicatives (chacun des pixels).

### (e) Affichez la 29e image à partir de la matrice X

Utilisez la fonction `imshow` pour la 29e ligne de la matrice X.

# 2. Analyse en composantes principales

Le but de cette section est de réduire la dimension du jeu de données d'entraînement, soit la matrice X. Nous ferons  une décomposition en valeurs singulières de cette matrice et nous récupérerons que les $k$ premières.

Les étapes sont les suivantes :
1. Centrer chacune des lignes de la matrice des visages d'entraînement pour obtenir la matrice $Z$.
2. Effectuer une décomposition en valeurs singulières de $Z$.
3. Illustrer le pourcentage de variabilité récupérée en fonction du nombre $k$ de composantes principales. 

## 2.1 Centrez les images de l'ensemble d'entraînement

### (a) Calculez le visage moyen $\bar{X}$ en faisait une moyenne de tous les visages pour chacun des pixels. Affichez le visage moyen avec la fonction `imshow`.

### (b) Calculez la matrice $Z$ centrée des visages de l'ensemble d'entraînement. 

Ensuite, afficher la différence entre le premier visage et le visage moyen avec la fonction `imshow`.

## 2.2 Décomposez en valeurs singulières la matrice $Z$.

### (a) Obtenez les matrices $U$ et $V$ ainsi que les valeurs singulières à l'aide de la fonction `svd`.

### (b) Affichez les premiers vecteurs singuliers de $V$ avec la fonction `imshow`. 

Ces composantes représentent les modes de plus grande variabilité. Dans la reconnaissance faciale, elles sont appelées les *eigenfaces*.

## 2.3 Illustrez le pourcentage de variabilité récupérée en fonction du nombre $k$ de composantes principales.

### (a) Tracez un graphique permettant de voir le pourcentage de la variance totale retenue en fonction du nombre de composantes principales.

### (b) Calculez le pourcentage de la variance récupérée en utilisant les $k$ premières composantes principales.

Utilisez les valeurs de k dans l'ensemble {8, 78, 196, 392} qui correspondent respectivement à des tailles de matrice ayant 1%, 10%, 25% et 50% de la taille de la matrice originale. 

# 3. Approximation des images avec les composantes principales

## 3.1 Approximation de la première image de l'ensemble d'entraînement

### (a) Calculez la combinaison linéaire des $k=8$ premières principales qui approxime le mieux l'image 1 de l'ensemble d'entraînement.

Pour ce faire, trouvez le meilleur plan qui recoupe les pixels de l'image 1. Indice : pensez à la régression linéaire.



### (b) Calculez l'approximation obtenue

### (c) Affichez l'image $zᵢ$ originale ainsi que son approximation

Vous pouvez ajouter le visage moyen pour une meilleure interprétation

## 3.2 Approximation du visage 1 avec les 78 premières composantes principales

Reprenez les étapes précédentes mais cette fois en utilisant les 78 premières composantes principales.

L'approximation à droite est obtenue avec une matrice de taille environ 10 fois moins volumineuse que le jeu de données original.

## 3.3 Approximation du visage 29 avec les 78 premières composantes principales

Reprenez les étapes précédentes mais cette fois avec le visage 29.

## 3.4 Approximation du visage 1 avec toutes les composantes principales

Reprenez les étapes précédentes mais cette fois en utilisant toutes les composantes principales.

## 3.5 Approximation de tous les visages de l'ensembles d'entraînement

En prenant les $k = 78$ premières composantes principales, calculez les pondérations η̂ᵢₖ permettant d'approximer tous les visages de l'ensemble d'entraînement. Construisez la matrice $H$ de taille $k \times n$ où la $i^e$ colonne correspond à la combinaison linéaire qui approxime l'image $i$ avec les $k$ premières composantes principales η̂ᵢₖ.


# 4. Reconnaissance faciale


L'idée de la reconnaissance faciale consiste à comparer le vecteur des coefficient $\mathbf{\hat{\eta}}$ de l'image à reconnaître avec les coefficients des images de l'ensemble d'entraînement. C'est une comparaison assez facile à faire car ce vecteur est de dimension raisonnable comparativement aux images originales. En effet, dans notre cas, si on prend 78 composantes principales, le vecteur des coefficients est un vecteur colonne de taille 78. On peut donc résumer toutes les images par leur vecteur des coefficients de taille 78. 

Pour savoir, si une nouvelle image représente une personne présente dans l'ensemble d'entraînement, on n'a qu'à comparer son vecteur des coefficients avec chacun des vecteurs des coefficients de l'ensemble d'entraînement. Si la différence entre les vecteurs est très grande, cela suggère que la personne est inconnue de l'ensemble d'entraînement. Si la différence est petite avec un des vecteurs de coefficients, cela suggère qu'il s'agit de la même personne. Le seuil doit être ajusté par une procédure de validation croisée.

Dans cette section, vous déciderez si une nouvelle image de l'ensemble test représente un personne connue ou inconnue. Vous le ferez en complétant les étapes suivantes :

1. Calculez les coefficients de la combinaison linéaire des colonnes de Vₖ approximant l'image inconnue zₒ.
2. Calculez la distance euclidienne entre les coefficients des images d'entraînement et celui de l'image de test.
3. Identifiez l'image de l'ensemble d'entraînement la plus proche de l'image de test.
4. En fonction de la distance, décidez si le visage se retrouve dans l'échantillon d'entraînement ou s'il est inconnu.

## 4.1 Calculez les coefficients de la combinaison linéaire des colonnes de Vₖ approximant  l'image inconnue zₒ.

Prenez $k = 78$ composantes.

### (a) Chargez une image de l'ensemble de test

In [ ]:
# jᵉ image de l'ensemble de test
j = 1

# chargement de la matrice des niveaux de gris
xₒ = imgrayconvert(testFileName[j], columnStack=true)

# Affichage de l'image brute
imshow(xₒ, (m₁, m₂))

### (b) Calculez la différence avec le visage moyen

### (c) Calculez les coefficients de la combinaison linéaires de Vₖ qui approxime le mieux zₒ

### (d) Calculez les vecteurs différences entre les coefficients de l'image et ceux de l'ensemble d'entraînement

### (e) Calculez la norme euclidienne de ces vecteurs différence

### (f) Identifiez l'image de l'ensemble d'entraînement pour laquelle la norme des différences est la plus petite.

Conservez la norme de la différence minimale.

### (g) Décidez si le visage se retrouve dans l'échantillon d'entraînement ou s'il est inconnu.

Si la distance minimale de la différence est suffisamment petite, alors on statuera que la personne est connue (elle se retrouve dans l'ensemble d'entraînement). Sinon, la personne est inconnue (elle ne se retrouve pas dans l'ensemble d'entraînement).

Il faut définir ce seuil à l'aide d'une procédure validation croisée. Par simplicité ici, utilisez le seuil de 3500².

Vous pouvez refaire les étapes du numéro 4 avec les autres images de l'ensemble de test et vous pouvez même tenter d'utiliser un nombre différent de composantes principales. Amusez-vous !  